##Реализация классификатора текста. 

Необходимо обучить модель для определения жанра фильма по его краткому описанию.

### Используем TensorFlow 2.0

Переключаемся на версию 2.0 (работает только в Colab)

In [0]:
%tensorflow_version 2.x

### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

import matplotlib.pyplot as plt
%matplotlib inline

### Загрузка и чтение данных

In [0]:
# подключение к google диску
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# рабочая директория; при первом запуске создадим директорию (если её еще не существует), 
# в противном случае надо заменить True на False 
if True:
    !mkdir "/content/drive/My Drive/Classificator_for_text"
%cd "/content/drive/My Drive/Classificator_for_text"

In [0]:
# загружаем данные (Genre_Classification_Dataset) в текущую рабочую директорию (Classificator_for_text)
if True:
    !7z x Genre_Classification_Dataset.7z

In [0]:
# посмотрим на содержимое файлов 
descript = open('Genre_Classification_Dataset/description.txt',mode='rt')
descript_res = descript.readlines()
descript_res

In [0]:
train_data = open('Genre_Classification_Dataset/train_data.txt',mode='rt')
train_data = train_data.readlines()
train_data[:3]

In [0]:
test_data = open('Genre_Classification_Dataset/test_data.txt',mode='rt')
test_data = test_data.readlines()
test_data[:3]

In [0]:
test_data_sol = open('Genre_Classification_Dataset/test_data_solution.txt',mode='rt')
test_data_sol = test_data_sol.readlines()
test_data_sol[:3]

### Предобработка данных

In [0]:
# преобразуем наши списковые данные в dataframe (трейновый датасет)

id_list = []
title = []
genre = []
descript = []

for line in train_data: 
    line_row = line.split(':::')
    id_list.append(line_row[0].strip())
    title.append(line_row[1].strip())
    genre.append(line_row[2].strip())
    descript.append(line_row[3].strip())

train_df = pd.DataFrame({'id': id_list, 'title': title, 
                         'genre': genre, 'descript': descript}, 
                        columns = ['id', 'title', 'genre', 'descript']) 

print(train_df.shape)
train_df.head()

In [0]:
# преобразуем наши списковые данные в dataframe (тестовый датасет)

id_list = []
title = []
genre = []
descript = []

for line in test_data_sol: 
    line_row = line.split(':::')
    id_list.append(line_row[0].strip())
    title.append(line_row[1].strip())
    genre.append(line_row[2].strip())
    descript.append(line_row[3].strip())

test_df = pd.DataFrame({'id': id_list, 'title': title, 
                        'genre': genre, 'descript': descript}, 
                       columns = ['id', 'title', 'genre', 'descript']) 

print(test_df.shape)
test_df.head()

In [0]:
# жанры и их количество
print('genres of train_df')
print(train_df['genre'].unique())
print(train_df['genre'].nunique())
print('genres of test_df')
print(test_df['genre'].unique())
print(test_df['genre'].nunique())

In [0]:
def genres_to_numb(dataframe): 
    dataframe['genre'] = dataframe['genre'].map({'drama': 0, 
                                                 'thriller': 1, 
                                                 'adult': 2, 
                                                 'documentary': 3, 
                                                 'comedy': 4, 
                                                 'crime': 5, 
                                                 'reality-tv': 6, 
                                                 'horror': 7, 
                                                 'sport': 8, 
                                                 'animation': 9, 
                                                 'action': 10, 
                                                 'fantasy': 11, 
                                                 'short': 12, 
                                                 'sci-fi': 13, 
                                                 'music': 14, 
                                                 'adventure': 15, 
                                                 'talk-show': 16, 
                                                 'western': 17, 
                                                 'family': 18, 
                                                 'mystery': 19, 
                                                 'history': 20, 
                                                 'news': 21, 
                                                 'biography': 22, 
                                                 'romance': 23, 
                                                 'game-show': 24, 
                                                 'musical': 25, 
                                                 'war': 26}).astype(int)

# genres of train_df в числа (27 категорий!)
genres_to_numb(train_df)

# genres of test_df в числа (27 категорий!)
genres_to_numb(test_df)

# pd.series to array
train_labels = train_df['genre'].to_numpy()
test_labels = test_df['genre'].to_numpy()

In [0]:
train_df.head(3)

In [0]:
test_df.head(3)

In [0]:
import collections
import re

# pd.series to array
train_data = train_df['descript'].to_numpy()
test_data = test_df['descript'].to_numpy()

# tokenizer and vocab
TOKEN_RE = re.compile(r'[\w\d]+')

def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

# токенизация корпуса 
def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

# добавление фейкового токена 
def add_fake_token(word2id, token='<PAD>'):
    word2id_new = {token: i + 1 for token, i in word2id.items()}
    word2id_new[token] = 0
    return word2id_new

# тексты в токены 
def texts_to_token_ids(tokenized_texts, word2id):
    return [[word2id[token] for token in text if token in word2id]
            for text in tokenized_texts]


def build_vocabulary(tokenized_texts, max_size=10000, max_doc_freq=0.8, 
                     min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq

# PAD_TOKEN = '__PAD__'
# NUMERIC_TOKEN = '__NUMBER__'
# NUMERIC_RE = re.compile(r'^([0-9.,e+\-]+|[mcxvi]+)$', re.I)

# def replace_number_nokens(tokenized_texts):
#     return [[token if not NUMERIC_RE.match(token) else NUMERIC_TOKEN for token in text]
#             for text in tokenized_texts]

In [0]:
train_tokenized = tokenize_corpus(train_data)
test_tokenized = tokenize_corpus(test_data)

print(' '.join(train_tokenized[0]))

In [0]:
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
                                             max_doc_freq=0.8, 
                                             min_count=5, 
                                             pad_word='<PAD>')

UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

In [0]:
plt.hist(word_doc_freq, bins=20)
plt.title('Распределение относительных частот слов')
plt.yscale('log');

In [0]:
# numbers of tokens
train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

print('\n'.join(' '.join(str(t) for t in sent)
                for sent in train_token_ids[:10]))

In [0]:
plt.hist([len(s) for s in train_token_ids], bins=20);
plt.title('Гистограмма длин предложений');

In [0]:
MAX_SEQ_LEN = 256 # Финальная длина последовательности

train_data = tf.keras.preprocessing.sequence.pad_sequences(
    train_token_ids,
    value=vocabulary["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

test_data = tf.keras.preprocessing.sequence.pad_sequences(
    test_token_ids,
    value=vocabulary["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

print("Length examples: {}".format([len(train_data[0]), len(train_data[1])]))
print('=====================================')
print("Entry example: {}".format(train_data[0]))

### Создание модели 

In [0]:
EMB_SIZE = 32 # Размер векторного представления (эмбеддинга) 
    
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(UNIQUE_WORDS_N, 32),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(27, activation=tf.nn.softmax),
])

model.summary()

### Подготовка модели к обучению


In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

### Разбиение на обучающую и валидационную выборку

In [0]:
# валидационная выборка составит около 25% 
border_split = int(train_df.shape[0]) - 13550

partial_x_train = train_data[:border_split]
x_val = train_data[border_split:]
partial_y_train = train_labels[:border_split]
y_val = train_labels[border_split:]

### Обучение модели

In [0]:
BATCH_SIZE = 256
NUM_EPOCHS = 26

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_val, y_val),
                    verbose=1)

### Оценка качества на тестовом датасете

In [0]:
results = model.evaluate(test_data, test_labels)

print('Test loss: {:.4f}'.format(results[0]))
print('Test accuracy: {:.2f} %'.format(results[1]*100))

1694/1694 [==============================] - 5s 3ms/step - loss: 1.6667 - acc: 0.5381
Test loss: 1.6667
Test accuracy: 53.81 %


Accuracy на тестовой выборке составляет больше 50%, что является очень хорошим результатом, т.к. у нас классификация идет на 27 категорий. 

### Графики лосса и точности на обучающем и валидационном датасетах

In [0]:
epochs = range(1, len(history.history['acc']) + 1)

plt.figure()
plt.plot(epochs, history.history['loss'], 'bo', label='Training loss')
plt.plot(epochs, history.history['val_loss'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

plt.figure()
plt.plot(epochs, history.history['acc'], 'bo', label='Training acc')
plt.plot(epochs, history.history['val_acc'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()